In [1]:
import pyspark.pandas as ps

from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.driver.memory", "8g")  # Set to your desired heap size

spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()


/Users/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/04 12:32:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load genotypes

In [2]:
vcfLikePath='./als_1kg.exon.num.txt'

genotypes = ps.read_csv(vcfLikePath, sep='\t')

/Users/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [3]:
genotypes.shape

(654394, 5925)

In [4]:
genotypes.columns

Index(['Gene', 'Func.knownGene', 'Func.refGene', 'Func.ensGene', 'ExonicFunc',
       'AAChange.knownGene', 'AAChange.refGene', 'AAChange.ensGene', 'chrom',
       'position',
       ...
       'CGND-HDA-02442', 'CGND-HDA-02438', 'CGND-HDA-02445', 'CGND-HDA-02693',
       'CGND-HDA-02446', 'CGND-HDA-02444', 'CGND-HDA-02439', 'CGND-HDA-02462',
       'CGND-HDA-02688', 'CGND-HDA-02440'],
      dtype='object', length=5925)

In [5]:
genotypes['CGND-HDA-02462'].unique()

0     0
1    NA
2     1
3    10
Name: CGND-HDA-02462, dtype: object

## Load gene sets

In [6]:
geneSets = {'alsod': ps.read_csv('./alsodList.csv', sep='\t')}

/Users/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [7]:
for set in geneSets:
    print(geneSets[set].head())

  Gene symbol                                             Gene name Category
0         AGT                                       angiotensinogen  Tenuous
1        ALAD                           aminolevulinate dehydratase  Tenuous
2        ALS2     alsin Rho guanine nucleotide exchange factor ALS2  Tenuous
3        ALS3  amyotrophic lateral sclerosis 3 (autosomal dominant)  Tenuous
4        ALS7                       amyotrophic lateral sclerosis 7  Tenuous


## Filter gene sets

In [11]:
filteredGeneSets = {}
for set in geneSets:
    filteredGeneSets[set] = genotypes[(genotypes['Gene'].isin(geneSets[set]['Gene symbol'].to_list()) & (genotypes['ExonicFunc'] == 'nonsynonymous_SNV'))]


/Users/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [12]:
filteredGeneSets['alsod'].shape

(3021, 5925)

In [15]:
filteredGeneSets['alsod'].spark.repartition(1).to_csv('./alsodGenotypes.csv', sep='\t', index=False)

/Users/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
23/10/04 13:43:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/10/04 13:43:47 WARN DAGScheduler: Broadcasting large task binary with size 1272.6 KiB
23/10/04 13:44:57 WARN DAGScheduler: Broadcasting large task binary with size 1272.6 KiB


23/10/04 18:16:29 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 816855 ms exceeds timeout 120000 ms
23/10/04 18:16:29 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/04 18:16:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$